In [1]:
from tournament import tournament
from reporting import Reporting
import importlib
from collections import namedtuple
from game_agent_comp import CustomPlayerComp
from value_functions import improved_score_fast_x2,improved_score_fast,\
    improved_score_fast_x3, partition_score_x2
from sample_players import null_score
from policy import SimplePolicy
import numpy as np
from copy import copy
#importlib.reload(reporting)

In [2]:
import pickle, glob

files = glob.glob('result*.pickle')
print(files)

depths = {}
for file in files[:3]:
    try:
        with open(file, 'rb') as handle:
            old_depths = pickle.load(handle)
            #print(old_depths)
    except:
        old_depths={}

    #print(old_depths)
    for d in old_depths:
        for p, v in d.items():
            if p not in depths:
                depths[p]=v
            else:
                depths[p] = depths[p]+v

#with open('result.pickle', 'wb') as handle:
#    pickle.dump(depths, handle)
print(depths.keys())



['result0.pickle', 'result1.pickle', 'result2.pickle', 'result3.pickle', 'result4.pickle', 'result5.pickle', 'result6.pickle', 'result7.pickle', 'result8.pickle', 'result9.pickle']
dict_keys(['Faster improved', 'improved, two steps exact, with reporting', 'improved, three steps exact'])


In [3]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

example = depths['improved, three steps exact'][0]
move = example[1]
print(move['allscores'])

from value_functions import to_index

def nice_allscores(x):
    scores =[score for score, cell in x]
    inds = [to_index(cell) for score, cell in x]
    return softmax(scores), inds
    
print(nice_allscores(move['allscores']))

[(-1.0, (0, 3)), (0.0, (3, 2)), (-1.0, (3, 0))]
(array([ 0.21194156,  0.57611688,  0.21194156]), [21, 17, 3])


In [4]:
# collate all moves in one big list
moves = []
final_moves =[]
#for player, games in depths.items():
games=depths['Faster improved']
for game in games:
    for move in game:
        if move['score'] != float('inf') and move['score'] != float('-inf'):
            moves.append(move)
        else:
            move_ = copy(move)
            if move['score'] == float('inf'):
                move_['score'] = 1
            else:
                move_['score'] = -1
            final_moves.append(move_)
            
            
        
print(len(moves), len(final_moves))
print(nice_allscores(moves[0]['allscores']))

7850 1605
(array([ 0.25,  0.25,  0.25,  0.25]), [23, 3, 21, 17])


In [ ]:
np.zeros([0,5])

array([], shape=(0, 5), dtype=float64)

In [ ]:
#%pdb

from neural import NNValueFunction, SelectionValueFunction
import numpy as np
from copy import copy
from sklearn.model_selection import train_test_split
import math 
import random

train_moves, test_moves = train_test_split(moves, test_size = 0.3)


val = SelectionValueFunction([5,5])

coeff = np.random.normal(size =val.coeff_len)*0.1
n = 0
while 53*n < len(coeff):
    coeff[53*n:(53*n+10)] = 0
    n+=1
    
val.set_coeff(coeff)



n = 0
train_diff = np.zeros(len(train_moves))
train_base = copy(train_diff)
test_diff = np.zeros(len(test_moves))
test_base = copy(test_diff)

while True:
    print('entering calibration next epoch...')
    dcoeff = np.zeros(val.coeff_len)
    random.shuffle(train_moves)
    for m, move in  enumerate(train_moves):
        scores, inds = nice_allscores(move['allscores'])
        diff = scores - val(input_vec = move['game'],
                        pos = move['pos'], indices = inds, mask = move['game'])
        train_diff[m] = np.linalg.norm(diff)
        train_base[m] = np.linalg.norm(scores - scores.mean())
        #square_print(nn(inp))
        #print(np.linalg.norm(diff))
        gr = val.nn.grad()
        #gr = gr /(1 + np.linalg.norm(gr))
        dcoeff += gr.dot(diff).transpose()[0]*0.01
        if m%100 ==0:
            #print(np.linalg.norm(dcoeff), np.linalg.norm(val.nn.get_coeff()))
            #dnorm = np.linalg.norm(dcoeff)
            #cnorm = np.linalg.norm(val.nn.get_coeff())
            #dcoeff = math.sqrt(cnorm/dnorm)*dcoeff
            if m%1000 == 0 and m>0:
                print(np.linalg.norm(dcoeff), np.linalg.norm(val.nn.get_coeff()))
                print('test: ',np.linalg.norm(train_diff)/np.linalg.norm(train_base))
            coeff = val.nn.get_coeff() + dcoeff
            #print(coeff.shape)
            val.nn.set_coeff(coeff)
            dcoeff = np.zeros(val.coeff_len)
        #if m%100 == 0
    
    for m,move in enumerate(test_moves):
        scores, inds = nice_allscores(move['allscores'])
        diff = scores - val(input_vec = move['game'],
                        pos = move['pos'], indices = inds)
        test_diff[m] = np.linalg.norm(diff)
        test_base[m] = np.linalg.norm(scores-scores.mean())
    print('train: ',np.linalg.norm(train_diff)/np.linalg.norm(train_base))
    print('test: ',np.linalg.norm(test_diff)/np.linalg.norm(test_base))


2 212
(212, 2)
1.21865559764e-40 -0.0
entering calibration next epoch...
0.000994863010134 4.46800424387
test:  0.998453634727
0.000864695438716 4.46808201831
test:  0.998141478513
0.000157950605399 4.46816719538
test:  0.998108443433
0.000966024285468 4.46821878482
test:  0.99815584624
0.000880396338428 4.46830847128
test:  0.998082503728
train:  0.998141748424
test:  0.996693861952
entering calibration next epoch...
0.00106376656697 4.46843482878
test:  0.998064092143
0.00040520253221 4.46856253511
test:  0.998354608201
0.000830179359571 4.46866073941
test:  0.998443352794
0.000506305877889 4.46882647324
test:  0.998461854452
0.000393206754928 4.46897536224
test:  0.998534590741
train:  0.998421064257
test:  0.997048746226
entering calibration next epoch...
0.000835114731941 4.46917791588
test:  0.998473641367


In [ ]:
from neural import NNValueFunction
import numpy as np
from copy import copy
from sklearn.model_selection import train_test_split

train_moves, test_moves = train_test_split(moves, test_size = 0.2)


val = NNValueFunction([1,3,3])
val.set_coeff(np.random.normal(size =val.coeff_len)*0.3)
n = 0
train_diff = np.zeros(len(train_moves))
train_base = copy(train_diff)
test_diff = np.zeros(len(test_moves))
test_base = copy(test_diff)
while True:
    for m, move in  enumerate(train_moves):
        train_diff[m] = move['score'] - val(input_vec = move['game'], pos = move['pos'])
        train_base[m] = move['score']
        #square_print(nn(inp))
        #print(np.linalg.norm(diff))
        gr = val.nn.grad()
        gr = gr /(1 + np.linalg.norm(gr))
        coeff = val.nn.get_coeff() + gr.dot(train_diff[m]).transpose()[0]*0.001
        #print(coeff.shape)
        val.nn.set_coeff(coeff)
        #if m%100 == 0
    for m,move in enumerate(test_moves):
        test_diff[m] = move['score'] - val(input_vec = move['game'], pos = move['pos'])
        test_base[m] = move['score']
    print('train: ',np.linalg.norm(train_diff)/np.linalg.norm(train_base))
    print('test: ',np.linalg.norm(test_diff)/np.linalg.norm(test_base))


In [ ]:
def transpose_list_of_lists(lol):
    
    transp = []

    for game in lol:
        for m, move_depth in enumerate(game):
            if len(transp) <= m:
                transp.append([])
            transp[m].append(move_depth)
    return transp



In [ ]:
#%pdb
from statistics import mean, stdev, median
import numpy as np

def get_mean_std(lol):
    result = {'mu':[], 'std':[],'median':[], 'strict_win':[], 'strict_loss':[]}
    
    for move in lol:
        if len(move) > 2:
            result['mu'].append(mean([m['depth'] for m in move]))
            result['std'].append(stdev([m['depth'] for m in move]))
            result['median'].append(median([m['depth'] for m in move]))
            result['strict_win'].append(\
                len([ m['score'] for m in move if m['score'] == float('inf')]) / len(move))
            result['strict_loss'].append(\
                len([ m['score'] for m in move if m['score'] == float('-inf')]) / len(move))
            
            
    result['mu'] = np.array(result['mu'] )
    result['std'] = np.array(result['std'])
    result['median'] = np.array(result['median'])
    result['strict_win'] = np.array(result['strict_win'])
    result['strict_loss'] = np.array(result['strict_loss'])
    return result

player_stats = {}
for p, d in depths.items():
    tmp = transpose_list_of_lists(d)
    #print(tmp)
    player_stats[p] = get_mean_std(transpose_list_of_lists(d))


In [ ]:
player_stats


In [ ]:
#import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure()
ax = plt.axes()
#ax.set_yscale("log")
lines = []
legend = []
n=0
lastpt = 12
for p, stats in player_stats.items():
    lines.append(\
        plt.errorbar(np.array(range(lastpt))+n,stats['mu'][0:lastpt], 
                     yerr=stats['std'][0:lastpt], fmt='o'))
    legend.append(p)
    n+= 0.1

plt.legend(lines,legend)
plt.show()

In [ ]:
plt.figure()
ax = plt.axes()
#ax.set_yscale("log")
lines = []
legend =[]
for p, stats in player_stats.items():
    lines.append(\
        plt.plot(range(len(stats['mu'])),stats['strict_win'] + stats['strict_loss']))
    legend.append(p)
plt.legend(legend)
plt.grid()
plt.show()

In [ ]:
plt.figure()
ax = plt.axes()
#ax.set_yscale("log")
lines = []
legend =[]
for p, stats in player_stats.items():
    lines.append(\
        plt.plot(range(len(stats['mu'])),stats['strict_win']-stats['strict_loss']))
    legend.append(p)
plt.legend(legend)
plt.grid()
plt.show()